#Dynamic Programming
How does that relate to RL? Finding the optimal Policy $\pi_*(s)$.

But what do we need?
* **Optimal Substracture**: Finding $\pi_*$ --------->1) Finding $\pi_*(S_1)$, 2) Finding $\pi_*(S_2)$, 3) Finding $\pi_*(S_3)$, .......

* **Overlapping Subproblems**: The solutions to the subproblems are **mutually dependent** $$V_*(s) = max_a Σ_{s',r}p(s',r|s,a)[r+\gamma v_*(s')]$$
$$q_*(s,a) = \Sigma_{s',r}p(s',r|s,a)[r+\gamma max_a' q_*(s',a')]$$

**So what does DP actually do?** \\
We setup a value table and keep track of state values and q values for each states. Based on the **Bellman Equation**, we do
$$V(s)←max_aΣ_{s',r}p(s',r|s,a)[r+\gamma v_*(s')]$$
We first sweep the state space and update the estimated value of the state, then we iteratively update the estimated state value to make more accurate estimation. \\

**Limitation** Need perfect model for DP, need access to these transition state probabilities  e.g. Rubik's Cube \\


#Our First DP algorithm: Value Iteration
**Goal:** Find the optimal policy $\pi_*$: $$\pi_*(s)=arg{max}_aΣ_{s',r}p(s',r|s,a)[r+\gamma v_*(s')]$$
$$\pi_*(s)←{max}_aΣ_{s',r}p(s',r|s,a)[r+\gamma v_*(s')]$$


Algorithm:




In [ ]:
# policy_prob initially random, state_value initially all 0,
# theta is the tolerance of delta, and gamma is the discounting factor
def value_iteration(policy_probs,state_values,theta=1e-6,gamma=0.99):
    delta = float('inf')

    # when V(s_n+1)-V(s_n) is less than tolerance, that means the estimated state value converges
    while delta > theta:
        delta = 0
        # for each s in S, check the value
        for row in range(5):
            for col in range(5):
                # record the current state value
                old_value = state_values[(row,col)]
                action_probs = None
                # estimate the maximum state value in next step
                max_qsa = float("-inf")

                for action in range(4):
                    # estimate the reward for next state if a certain action has been taken
                    next_state,reward,_,_ = env.simulate_step((row,col),action)
                    qsa = reward + gamma * state_values[next_state]

                    # since this is value iteration, there's always an optimal action for certain state,
                    # so we set the possibility into 1, we only choose the action maximizing the return
                    if qsa > max_qsa:
                        max_qsa = qsa
                        action_probs = np.zeros(4)
                        action_probs[action] = 1.

                state_values[(row,col)] = max_qsa
                policy_probs[(row,col)] = action_probs

                delta = max(delta,abs(max_qsa - old_value))

#Another DP algorithm: Policy Iteration
**Policy Iteration: A process that alternatively improves the Estimated Values and the Policy ** \\
Equation: $$V(s)← Σ_a π(s|a) Σ_{s',r}p(s',r|s,a)[r+\gamma v_*(s')]$$

**PopUp: What is q-function?**
$$q_π(s,a)=Σ_{s',r}P(s',r|s,a)[r+γv_π(s')]$$
Given current state s and action a, return the sum of probabilities of arriving next state s' after executing the certain action, times the reward obtained by arriving that state plus the discounted value of that state. \\
Each action has different q-value, and we need to compare different q-values and modify the policy respectively based on larger q-value

**Policy Improvement Theorem** \\
if $$q_π(s,π'(s)) \geq v_π(s)$$, then $$v_{π'}(s) \geq v_π(s)$$


**Algorithm:**
* Policy Evaluation --- Evaluate the current state value according to policy
* Policy Improvement --- Use the evaluated state value to modify policy


In [ ]:
def policy_evaluation(policy_probs,state_values,theta=1e-6,gamma=0.99):
    delta = float('inf')
    while delta > theta:
        delta = 0

        # For each s in S(n)
        for row in range(5):
            for col in range(5):
                old_value = state_values[(row,col)]
                new_value = 0.
                # The probabilities for taking certain action in current state s
                action_probabilities = policy_probs[(row,col)]

                # For current state, estimating every rewards in the next step, weight them
                # by probabilities of each action, and add them up into the state value V(s)
                for action,prob in enumerate(action_probabilities):
                    next_state,reward,_,_ = env.simulate_step((row,col),action)
                    new_value += prob * (reward + gamma * state_values[next_state])

                state_values[(row,col)] = new_value

                delta = max(delta,abs(old_value-new_value))

In [ ]:
def policy_improvement(policy_probs,state_values,gamma=0.99):

    policy_stable = True

    for row in range(5):
        for col in range(5):
            # the current action with highest probability
            old_action = policy_probs[(row,col)].argmax()

            new_action = None
            max_qsa = float('-inf')

            for action in range(4):
                next_state,reward,_,_ = env.simulate_step((row,col),action)
                qsa = reward + gamma * state_values[next_state]

                if qsa > max_qsa:
                    new_action = action
                    max_qsa = qsa

            action_probs = np.zeros(4)
            action_probs[new_action] = 1.
            policy_probs[(row,col)] = action_probs

            if new_action != old_action:
                policy_stable = False

    return policy_stable

In [ ]:
def policy_iteration(policy_probs,state_values,theta=1e-6,gamma=0.99):
    policy_stable = False

    while not policy_stable:

        policy_evaluation(policy_probs,state_values,theta,gamma)
        plot_values(state_values,frame)

        policy_stable = policy_improvement(policy_probs,state_values,gamma)
        plot_policy(policy_probs,frame)

#But what if the Environment becomes a Black Box? Would DP still applicable?

#General Policy Iteration


#Monte Carlo Methods
Family of methods that learn optimal v_*(s) or q_*(s,a) values based on Samples \\

They approximate the values by interacting with the environment to generate sample returns and averaging them \\

#Advantages
* The estimate of one state does not depend on the rest
* The cost of estimating the state value is independent of the total number of states
* No need a model of Environment

#Stochastic Policies
* On-policy Learning: Generate samples using the same policy $π$ that we are using to optimize
* Off-policy Learning: Generate samples with exploratory policy b different from the one we are going to optimize


**On Policy Learning** \\
$\epsilon$-greedy policy: With probability $ϵ$ select a random action, and with probability $1-ϵ$ select the action with highest Q(s,a)

Action is optimal $$π(a|s) = 1-ϵ+ϵ_r (when a = a^*)$$
Action not optimal $$π(a|s) = ϵ_r (when a \neq a^*)$$

Probability of choosing sub-optimal based on estimation of q-value action: $$ϵ_r=\frac{ϵ}{|A|}$$


#Algorithm:


In [ ]:
def on_policy_mc_control(policy,action_values,episodes,gamma=0.99,epsilon=0.2):

    sa_returns = {} # state-action pair

    for episode in range(1,episodes+1):
        state = env.reset()
        done = False
        transitions = []

        while not done:
            action = policy(state,epsilon)
            next_state, reward, done, _ = env.step(action)
            transitions.append([state,action,reward])
            state = next_state

        G = 0

        for state_t, action_t, reward_t in reversed(transitions):
            G = reward_t + gamma * G

            if not (state_t, action_t) in sa_returns:
                sa_returns[(state_t,action_t)] = []

            sa_returns[(state_t,action_t)].append(G)
            action_values[state_t][action_t] = np.mean(sa_returns[state_t,action_t])